In [22]:
import os
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
from pathlib import Path
import cv2
import mmcv
from tqdm import tqdm

C:\Users\Shaheryar\anaconda3\envs\sord_env\lib\site-packages\mmcv\__init__.py:21: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  'On January 1, 2023, MMCV will release v2.0.0, in which it will remove '


In [16]:
!pip3 install tqdm

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)


In [2]:
DATASET_BASE_PATH=Path('D:\SORD AI\Dataset')

In [3]:
# For coco dataset
# There are three necessary keys in the json file:

# images: contains a list of images with their information like file_name, height, width, and id.
# annotations: contains the list of instance annotations.
# categories: contains the list of categories names and their ID.


# {
#     "images": [image],
#     "annotations": [annotation],
#     "categories": [category]
# }


# image = {
#     "id": int,
#     "width": int,
#     "height": int,
#     "file_name": str,
# }
# annotation = {
#     "id": int,
#     "image_id": int,
#     "category_id": int,
#     "area": float,
#     "bbox": [x,y,width,height],
#     "iscrowd": 0 or 1,
# }
# categories = [{
#     "id": int,
#     "name": str,
#     "supercategory": str,
# }]

### Generating annotations

In [4]:
f= open(DATASET_BASE_PATH/'eval'/'objectclasses.json','r')
labels_info = json.load(f)

In [5]:
categories=[]
for cat in labels_info:
    categories.append({
        'name':cat['Name'],
        'id':cat['Id'],
    })

In [6]:
categories

[{'name': 'l_klt_4147', 'id': 1011},
 {'name': 'l_klt_6147', 'id': 1012},
 {'name': 'l_klt_8210', 'id': 1013},
 {'name': 'pallet', 'id': 1100},
 {'name': 'str', 'id': 2050},
 {'name': 'cabinet', 'id': 1040},
 {'name': 'locker', 'id': 1030},
 {'name': 'jack', 'id': 1120},
 {'name': 'dolly', 'id': 1110},
 {'name': 'spring_post', 'id': 1135},
 {'name': 'exit_sign', 'id': 4000},
 {'name': 'fire_extinguisher', 'id': 5010},
 {'name': 'stillage_open', 'id': 1003},
 {'name': 'stillage_close', 'id': 1002},
 {'name': 'cardboard_box', 'id': 1070},
 {'name': 'forklift', 'id': 2010},
 {'name': 'bicycle', 'id': 2000}]

In [18]:
images=[]
annotations=[]
for folder in os.listdir(DATASET_BASE_PATH/'SORD_2022'):
    print(folder)
    img_id_prefix=''
    for x in (folder.split('_')[0:-1]):
        img_id_prefix=img_id_prefix+x
        img_id_prefix = img_id_prefix+'_'
    img_id_prefix = img_id_prefix[0:-1] 
    
    folder_imgs=os.listdir(os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'images'))
     if '360p' in folder:
            H,W,C=(720,1080,3)
        else:
            H,W,C=(360,640)
            
    for i in tqdm(range(len(folder_imgs))):
        img_name=folder_imgs[i]
        file_name=os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'images',img_name)
        img_id=img_id_prefix+'_'+img_name.split('.')[0]
        images.append({
            'id':img_id,
            'file_name':file_name,
            'height':H,
            'width':W
        })
        
        f= open(os.path.join(DATASET_BASE_PATH,'SORD_2022',folder,'labels','json',img_name.replace('jpg','json')))
        file_annotations=json.load(f)
        for annot_content in file_annotations:
            x=annot_content['Left']
            y=annot_content['Top']
            width=abs(annot_content['Right']-annot_content['Left'])
            height=abs(annot_content['Bottom']-annot_content['Top'])

            annotations.append({
                "id": img_id+'_'+str(annot_content['Id']),
                "image_id": img_id,
                "category_id": annot_content['ObjectClassId'],
                "bbox": [x, y, width, height],
                'area':width*height
            })

        


SORDI_2022_h4001_bicycle


100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 2597.55it/s]


SORDI_2022_h4002_cabinet


100%|████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:00<00:00, 2589.36it/s]


SORDI_2022_h4003_cardboard box


...

100%|████████████████████████████████████████████████████████████████████████████| 17301/17301 [08:19<00:00, 34.67it/s]


SORDI_2022_h4024_regensburg plant 360p


100%|████████████████████████████████████████████████████████████████████████████| 58289/58289 [38:12<00:00, 25.43it/s]


SORDI_2022_h4025_regensburg plant


100%|████████████████████████████████████████████████████████████████████████████| 47550/47550 [28:50<00:00, 27.48it/s]


In [19]:
print('Size of images',len(images))
print('Size of Annotations',len(annotations))

Size of images 194337
Size of Annotations 13058467


In [23]:
coco_format_json = dict(
        images=images,
        annotations=annotations,
        categories=categories)
mmcv.dump(coco_format_json, 'all_images_coco.json')